# Risk categories

In [2]:
!pip install --upgrade pip
!pip install pandas

In [4]:
import numpy as np
import pandas as pd
import pickle
import datetime

# import custom functions
import sys
sys.path.append('../../onc_functions')

from risk import get_risk_categories

print('python-' + sys.version)
dte = datetime.datetime.now()
dte = dte.strftime("%Y%m%d")

python-3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


In [5]:
#load fairness details
with open('../complete_fairness_data.pickle', 'rb') as f:
        dataset = pickle.load(f)
dataset.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,2969676.0,0,72.0,2.0,1.0,2.0,2.0,8
1,2969869.0,1,73.0,2.0,1.0,2.0,2.0,9
2,2970021.0,0,55.0,2.0,1.0,1.0,1.0,8
3,2970067.0,0,72.0,1.0,1.0,1.0,2.0,9
4,2970686.0,0,49.0,2.0,1.0,2.0,2.0,8


In [2]:
# load pooled results from the xgboost imputed model (not calibrated)

pred_df = pd.read_csv('./roc_results/2021_xgb_pooling_results_final_roc.csv')
pred_df = pred_df.loc[:,['averaged','died_in_90','usrds_id']].copy()
pred_df.head()

,averaged,died_in_90,usrds_id
0,0.467093,1,31089.0
1,0.473195,0,34521.0
2,0.290782,0,46751.0
3,0.225663,0,50506.0
4,0.041878,0,54985.0


In [6]:
# merge the 2 dataframes
data = pred_df.merge(dataset, how='left', on=['usrds_id','died_in_90'])
data.head()

,averaged,died_in_90,usrds_id,inc_age,sex,dialtyp,race,hispanic,subset
0,0.467093,1,31089.0,80.0,1.0,1.0,2.0,2.0,9
1,0.473195,0,34521.0,64.0,2.0,1.0,1.0,2.0,8
2,0.290782,0,46751.0,69.0,2.0,1.0,1.0,2.0,8
3,0.225663,0,50506.0,73.0,2.0,NaN,1.0,2.0,9
4,0.041878,0,54985.0,48.0,1.0,1.0,1.0,2.0,8


In [8]:
risk_cat = get_risk_categories(data,
                               y_proba_col_name='averaged',
                               y_true_col_name='died_in_90')
risk_cat

,Risk Category,Count,Count Died in 90,Count Survived,Percent Died in 90
0,0-0.09,149202,1681,147521,0.011267
1,0.1-0.19,76283,3315,72968,0.043457
2,0.2-0.29,44122,3784,40338,0.085762
3,0.3-0.39,29109,4082,25027,0.140232
4,0.4-0.49,20156,3946,16210,0.195773
5,0.5-0.59,13145,3424,9721,0.260479
6,0.6-0.69,7775,2699,5076,0.347138
7,0.7-0.79,3919,1753,2166,0.447308
8,0.8-0.89,1246,755,491,0.605939
9,0.9-0.99,343,294,49,0.857143


In [10]:
#save 
risk_cat.to_csv('./roc_results/' + str(dte) + '_xgb_imputed_risk_cat.csv')